In [48]:
import numpy as np

In [49]:
def get_dict(data):
  words = sorted(list(set(data)))
  n = len(words)
  idx = 0
  word2Ind = {}
  Ind2word = {}
  for k in words:
    word2Ind[k] = idx
    Ind2word[idx] = k
    idx += 1
  return word2Ind, Ind2word

In [50]:
# Define the size of word embedding vectors
N = 3

# Define the size of vocabulary
V = 5

In [51]:
# Initialization of weights and biases
# Define the first matrix of weights
W1 = np.array([[ 0.41687358,  0.08854191, -0.23495225,  0.28320538,  0.41800106],
               [ 0.32735501,  0.22795148, -0.23951958,  0.4117634 , -0.23924344],
               [ 0.26637602, -0.23846886, -0.37770863, -0.11399446,  0.34008124]])

# Define second matrix of weigths
W2 = np.array([[-0.22182064, -0.43008631,  0.13310965],
               [ 0.08476603,  0.08123194,  0.1772054 ],
               [ 0.1871551 , -0.06107263, -0.1790735 ],
               [ 0.07055222, -0.02015138,  0.36107434],
               [ 0.33480474, -0.39423389, -0.43959196]])

# Define first vector of biases
b1 = np.array([[ 0.09688219],
               [ 0.29239497],
               [-0.27364426]])

# Define second vector of biases
b2 = np.array([[ 0.0352008 ],
               [-0.36393384],
               [-0.12775555],
               [-0.34802326],
               [-0.07017815]])

In [52]:
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of W1: {W1.shape} (NxV)')
print(f'size of b1: {b1.shape} (Nx1)')
print(f'size of W2: {W2.shape} (VxN)')
print(f'size of b2: {b2.shape} (Vx1)')

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of W1: (3, 5) (NxV)
size of b1: (3, 1) (Nx1)
size of W2: (5, 3) (VxN)
size of b2: (5, 1) (Vx1)


In [53]:
# Define the tokenized version of the corpus
words = ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']

# Get 'word2Ind' and 'Ind2word' dictionaries for the tokenized corpus
word2Ind, Ind2word = get_dict(words)

# Define the 'get_windows' function
def get_windows(words, C):
  i = C
  while i < len(words) - C:
    center_word = words[i]
    context_words = words[(i - C) : i] + words[(i + 1) : (i + C + 1)]
    yield context_words, center_word
    i += 1

# Define the 'word_to_one_hot_vector'
def word_to_one_hot_vector(word, word2Ind, V):
  one_hot_vector = np.zeros(V)
  one_hot_vector[word2Ind[word]] = 1
  return one_hot_vector

# Define the 'context_words_to_vector' function
def context_words_to_vector(context_words, word2Ind, V):
  context_words_vector = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
  context_words_vector = np.mean(context_words_vector, axis = 0)
  return context_words_vector

# Define the generator function 'get_training_examples'
def get_training_example(words, C, word2Ind, V):
  for context_words, center_word in get_windows(words, C):
    yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

In [54]:
# Save generator object in the 'training_examples' variable with the desired arguments
training_examples = get_training_example(words, 2, word2Ind, V)

In [55]:
# Get the first value from genrator
x_array, y_array = next(training_examples)

In [56]:
# Print the context words vector
x_array

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [57]:
# Print one hot vector of center word
y_array

array([0., 0., 1., 0., 0.])

In [58]:
# Copy vector
x = x_array.copy()

# Reshape it
x.shape = (V,1)

# Print it
print(f'x:\n{x}\n')

# Copy vector
y = y_array.copy()

# Reshape it
y.shape = (V,1)

# Print it
print(f'y:\n{y}')

x:
[[0.25]
 [0.25]
 [0.  ]
 [0.5 ]
 [0.  ]]

y:
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


In [59]:
# Define the "relu" function
def relu(z):
  result = z.copy()
  result[result < 0] = 0
  return result

# Define softmax function
def softmax(z):
  e_z = np.exp(z)
  sum_e_z = np.sum(e_z)
  return e_z / sum_e_z